# Введение

## Описание задачи
[источник](https://contest.yandex.ru/contest/20144/problems/)

### RUS

Каждый пользователь, размещая объявление об аренде квартиры, хочет понимать, сколько времени потребуется для сдачи объекта. В первую очередь такая информация нужна на форме подачи объявлений, что заметно ограничивает набор возможных признаков. Вам предлагается построить модель, прогнозирующую длительность экспозиции объявлений на Яндекс.Недвижимости.
Для построения целевой переменной срок экспозиции разбит на несколько классов, каждому из которых соответствует целое число: "меньше 7 дней"(1), "7-14 дней"(2), "15-30 дней"(3), "30-70 дней"(4), "более 70 дней"(5).
Метрика, по которой оцениваются решения, записывается следующим образом:
$$
metric = \frac{1}{l}\sum\limits_{i=1}^l exp^{[prediction_i = target_i]}
$$

При отправке решения метрика вычисляется на публичной части тестовой выборки. Финальное значение будет определено по скрытой части тестовой выборки, чтобы исключить переобучение. Это значение будет опубликовано после окончания хакатона.
Оценивается только последняя посылка. Убедитесь, что на момент окончания хакатона ваша последняя посылка содержит именно тот прогноз, для которого вы хотите, чтобы было посчитано финальное значение метрики на скрытой части тестовой выборки.

## Описание датасета

### RUS

* **id** - id объявления про аренду квартиры
* **building_id** - id дома
* **unified_address** - адрес дома
* **building_series_id** - id серии дома
* **build_year** - год постройки дома
* **site_id** - id жилого комплекса, если у недавно построенных домов
* **parking** - тип парковки дома
* **expect_demolition** - дом входит в программу реновации и ожидает сноса
* **flats_count** - количество квартир в доме	
* **building_type** - тип стен в доме
* **main_image** - часть урла главной фотографии в объявлении, добавив "http:" можно получить урл (http://avatars.mds.yandex.net/get-realty/1702013/add.d352c435e10b2c47092f43a332bedb13.realty-api-vos/main/)
* **latitude, longitude** - координаты дома
* **total_area** - площадь квартиры в кв. м.
* **ceiling_height** - высота потолков в квартире
* **rooms** - количество комнат в квартире
* **floors_total** - количество этажей в доме
* **floor** - этаж квартиры
* **living_area** - жилая площадь в квартире
* **kitchen_area** - площадь кухни
* **is_apartment** - квартира юридически оформлена как апартаменты
* **studio** - квартиры является студией
* **has_elevator** - наличие лифта в доме	
* **day** - первый день экспозиции квартиры
* **balcony** - тип и наличие балкона
* **renovation** - качество ремонта
* **lolality_name** - имя населенного пункта
* **price** - цена аренды квартиры за 1 месяц
* **target** - срок экспозиции, который надо научиться предсказывать
* **target_string** - строковое представление срока экспозиции

# Подготовка ноутбука

## Первичные константы 

In [1]:
PROJECT_NAME = "hack_the_realty_exposition"
MOUNT_DIR = '/content/drive' # In case Colab Usage
VALIDATE_RATIO = 0.2

## Дополнительные установки

In [2]:
!pip install catboost

     |████████████████████████████████| 66.1MB 58kB/s 


## Библиотеки

In [3]:
import os

from datetime import datetime

from collections import Counter

import numpy as np

import pandas as pd

import catboost as ctb
from catboost import CatBoostClassifier, Pool
from catboost.utils import get_roc_curve

from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import roc_auc_score

from plotly.subplots import make_subplots
import plotly.graph_objects as go

import matplotlib.pyplot as plt

%matplotlib inline

## Обработка случая работы в Google.Colab

### Подключение библиотек

In [4]:
try:
    from google.colab import files, drive
    
    USE_COLAB = True
except:
    USE_COLAB = False

if USE_COLAB:
    print("Don't forget to avoid disconnections:")
    print("""
function ClickConnect(){
    console.log("Clicking"); 
    document.querySelector("colab-connect-button").click() 
}
setInterval(ClickConnect,60000)
    """)

Don't forget to avoid disconnections:

function ClickConnect(){
    console.log("Clicking"); 
    document.querySelector("colab-connect-button").click() 
}
setInterval(ClickConnect,60000)
    


### Подключение к Google.Drive

In [5]:
if USE_COLAB:
    drive.mount(MOUNT_DIR)
    DRIVE_DIR = os.path.join(MOUNT_DIR, 'My Drive')
    print(f"Drive directory is {DRIVE_DIR}")

Mounted at /content/drive
Drive directory is /content/drive/My Drive


## Объявление рабочей директории

Подключение к Google.Drive в случае работы c Google.Colab

In [6]:
PROJECT_DIR = os.path.join(DRIVE_DIR, 'projects', PROJECT_NAME) if USE_COLAB else './'
WORK_DIR = '/content' if USE_COLAB else PROJECT_DIR
print(f"Project directory is {PROJECT_DIR}")
print(f"Working directory is {WORK_DIR}")

Project directory is /content/drive/My Drive/projects/hack_the_realty_exposition
Working directory is /content


# Обработка данных

На выходе должны быть объявлены переменные:

* **X_train**: pd.DataFrame\
Id | ClientID | ClientType | CoachID | GymID | TrainingID | Month | Day
* **y_train**: pd.Series\
Id | Target
* **X_valid**: pd.DataFrame\
Id | ClientID | ClientType | CoachID | GymID | TrainingID | Month | Day
* **y_valid**: pd.Series\
Id | Target
* **X_test**: pd.DataFrame\
Id | ClientID | ClientType | CoachID | GymID | TrainingID | Month | Day

TODO: Обновить

## Загрузка

Загрузка train/test датасета в датафреймы

На выходе должны быть объявлены две переменные:
* train_dataset: pd.Dataframe
* test_dataset: pd.Dataframe

### Объявление путей

In [12]:
src_data_dir_path = os.path.join(PROJECT_DIR, 'data')
dist_data_dir = os.path.join(WORK_DIR, 'data')

data_archive_path = os.path.join(src_data_dir_path, 'E.zip')

### Разархивация

In [13]:
!unzip -o "$data_archive_path" -d "$dist_data_dir" > /dev/null

### Считывание

In [19]:
train_file_path = os.path.join(dist_data_dir, 'E', 'exposition_train.tsv')
test_file_path = os.path.join(dist_data_dir, 'E', 'exposition_test.tsv')

train_dataset = pd.read_csv(train_file_path, index_col='id', sep='\t')
test_dataset = pd.read_csv(test_file_path, index_col='id', sep='\t')

## Визуализация

### Просмотр первых строк

In [26]:
train_dataset.head()

,building_series_id,site_id,target,parking,target_string,build_year,expect_demolition,main_image,latitude,total_area,ceiling_height,rooms,floors_total,living_area,floor,is_apartment,building_id,has_elevator,studio,unified_address,area,kitchen_area,day,longitude,price,flats_count,building_type,balcony,locality_name,renovation
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5677548107212057955,1564812,0,1,OPEN,LESS_7,2005,False,//avatars.mds.yandex.net/get-realty/903734/add...,55.645313,105.000000,3.00,3,20,50.0,14,False,7969879732878112812,True,False,"Россия, Москва, Пролетарский проспект, 7",105.000000,15.0,2018-07-15,37.657490,95000,407,MONOLIT,BALCONY,Москва,EURO
155646401125694364,1564812,0,2,CLOSED,7_14,2010,False,//avatars.mds.yandex.net/get-realty/1702013/ad...,55.537102,40.000000,3.00,1,3,0.0,1,False,7667415960903930340,False,False,"Россия, Москва, посёлок Первомайское, Централь...",40.000000,10.0,2019-01-18,37.155632,25000,40,MONOLIT,UNKNOWN,посёлок Первомайское,COSMETIC_DONE
9186198458182518100,663302,0,2,OPEN,7_14,1995,False,//avatars.mds.yandex.net/get-realty/924080/add...,55.662956,37.599998,2.64,0,17,0.0,4,False,7166215405310646476,True,True,"Россия, Москва, улица Намёткина, 13к1",37.599998,0.0,2018-04-24,37.555466,26000,472,PANEL,LOGGIA,Москва,GOOD
10844743366553352344,1564812,0,2,OPEN,7_14,2018,False,//avatars.mds.yandex.net/get-realty/1521999/ad...,55.669151,80.000000,0.00,3,27,49.0,23,False,2039402855860137453,True,False,"Россия, Московская область, Одинцово, Верхне-П...",80.000000,20.0,2019-02-19,37.285000,35000,156,PANEL,UNKNOWN,Одинцово,GOOD
3712912186792420056,1564812,0,3,UNKNOWN,14_30,2004,False,//avatars.mds.yandex.net/get-realty/50286/f5c8...,55.828518,100.000000,3.00,3,4,0.0,3,False,4638454967482853510,True,False,"Россия, Москва, улица Рословка, 12к1",100.000000,0.0,2017-08-08,37.361897,80000,31,MONOLIT,UNKNOWN,Москва,EURO


In [33]:
test_dataset.head()

,building_series_id,site_id,parking,build_year,expect_demolition,main_image,latitude,total_area,ceiling_height,rooms,floors_total,living_area,floor,is_apartment,building_id,has_elevator,studio,unified_address,area,kitchen_area,day,public,longitude,price,flats_count,building_type,balcony,locality_name,renovation
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
13762887891614807236,663294,0,UNKNOWN,1971,False,//avatars.mds.yandex.net/get-realty/1900763/ad...,55.795704,36.000000,2.64,1,12,0.000000,10,False,1470199257376425951,True,False,"Россия, Москва, Стрелецкая улица, 8",36.000000,0.0,2020-01-25,True,37.602478,40000,80,PANEL,UNKNOWN,Москва,UNKNOWN
14654451946329972059,712125,0,UNKNOWN,1986,False,//avatars.mds.yandex.net/get-realty/1583116/ad...,55.605583,40.000000,2.48,1,16,20.000000,9,False,2797222858463679248,True,False,"Россия, Москва, Ясеневая улица, 36/2",40.000000,10.0,2019-11-19,True,37.743679,25000,222,PANEL,LOGGIA,Москва,COSMETIC_DONE
17449292585625593873,0,0,UNKNOWN,2014,False,//avatars.mds.yandex.net/get-realty/2124710/ad...,55.925560,25.000000,0.00,0,16,12.000000,16,False,3589071626016088122,True,True,"Россия, Московская область, Королёв, улица Мар...",25.000000,0.0,2020-01-11,True,37.862965,19000,179,MONOLIT,LOGGIA,Королёв,COSMETIC_DONE
15597282206699587329,0,0,UNKNOWN,2001,False,//avatars.mds.yandex.net/get-realty/2958378/ad...,55.432522,42.000000,0.00,1,10,20.000000,4,False,7320605563669842994,True,False,"Россия, Московская область, Подольск, Комсомол...",42.000000,10.0,2020-01-27,True,37.544224,20000,0,PANEL,LOGGIA,Подольск,COSMETIC_DONE
3718201047023531068,1564812,0,UNKNOWN,2019,False,//avatars.mds.yandex.net/get-realty/2732616/ad...,55.917530,73.300003,2.80,3,16,45.799999,6,False,3672377569291994545,True,False,"Россия, Московская область, Химки, улица Герма...",73.300003,10.2,2020-03-04,False,37.411098,68000,0,MONOLIT,TWO_LOGGIA,Химки,EURO


### Изучение различных полей

* total_area и area

In [52]:
assert len(train_dataset[train_dataset.total_area != train_dataset.area]) == 0
assert len(test_dataset[test_dataset.total_area != test_dataset.area]) == 0

print("Поля total_area и area дубликаты")

Поля total_area и area дубликаты


* day

In [53]:
print(train_dataset['day'].min())
print(train_dataset['day'].max())

2017-01-01
2019-10-31


In [54]:
print(test_dataset['day'].min())
print(test_dataset['day'].max())

2019-11-01
2020-03-31


In [38]:
train_dataset.columns

Index(['building_series_id', 'site_id', 'target', 'parking', 'target_string',
       'build_year', 'expect_demolition', 'main_image', 'latitude',
       'total_area', 'ceiling_height', 'rooms', 'floors_total', 'living_area',
       'floor', 'is_apartment', 'building_id', 'has_elevator', 'studio',
       'unified_address', 'area', 'kitchen_area', 'day', 'longitude', 'price',
       'flats_count', 'building_type', 'balcony', 'locality_name',
       'renovation'],
      dtype='object')

## Финальная обработка

### Извлечение выборки и таргета

На выходе должны быть объявлены переменные:

* **X_train_pure**: pd.DataFrame\
id | building_series_id | site_id | parking | build_year | expect_demolition | latitude | total_area | ceiling_height | rooms | floors_total | living_area | floor | is_apartment | building_id | has_elevator | studio | unified_address | area | 
* **y_train**: pd.Series\
Id | 
* **X_valid_pure**: pd.DataFrame\
Id | 
* **y_valid**: pd.Series\
Id | 
* **X_test_pure**: pd.DataFrame\
Id | 

In [ ]:
train_dataset = train_dataset.sort_values(by='Time')

partition = int(len(train_dataset) * (1 - VALIDATE_RATIO))
train_train_dataset = train_dataset.iloc[:partition]
train_valid_dataset = train_dataset.iloc[partition:]

X_train_train_pure = train_train_dataset.drop('Target', axis=1)
y_train_train = train_train_dataset['Target']

X_train_valid_pure = train_valid_dataset.drop('Target', axis=1)
y_train_valid = train_valid_dataset['Target']
X_test_pure = test_dataset

### Добавление дополнительных признаков

* **X_train**: pd.DataFrame\
Id | ClientID | ClientType | CoachID | GymID | TrainingID | Month | Day | DayOfWeek | Season
* **y_train**: pd.Series\
Id | Target
* **X_valid**: pd.DataFrame\
Id | ClientID | ClientType | CoachID | GymID | TrainingID | Month | Day | DayOfWeek | Season
* **y_valid**: pd.Series\
Id | Target
* **X_test**: pd.DataFrame\
Id | ClientID | ClientType | CoachID | GymID | TrainingID | Month | Day | DayOfWeek | Season

In [ ]:
def add_time_features(X):
    datetime_series = pd.to_datetime(X['Time'], format='%Y-%m-%d %H:%M:%S')
    X['Month'] = pd.DatetimeIndex(datetime_series).month
    X['Day'] = pd.DatetimeIndex(datetime_series).day
    X['DayOfWeek'] = pd.DatetimeIndex(datetime_series).dayofweek
    X['Season'] = (X['Month'] % 12 + 3) // 3
    return X

def prepare_sample(X):
    X = X.copy()
    X = add_time_features(X)
    return X.drop('Time', axis=1)

In [ ]:
X_train_train = prepare_sample(X_train_train_pure)
X_train_valid = prepare_sample(X_train_valid_pure)
X_test = prepare_sample(X_test_pure)

# Эксперименты с обучением

На выходе должны быть объявлены переменные

* **y_test_predicted**: pd.Series \
Id | Target
* **SUBMISSTION_FILE_NAME**: string\
Названия файла для сохранения

## Catboost

### Объявление пула и параметров

In [ ]:
cat_features = [
    'ClientID',
    'ClientType',
    'CoachID',
    'GymID',
    'TrainingID',
    'Month',
    'Day',
    'DayOfWeek',
    'Season'
]
ignored_features = [
]
train_train_pool = Pool(
    X_train_train,
    y_train_train,
    cat_features=cat_features
)
train_valid_pool = Pool(
    X_train_valid,
    y_train_valid,
    cat_features=cat_features
)
params = {
    'iterations': 400,
    'learning_rate': 0.08,
    'eval_metric': 'AUC',
#     'random_seed': 113,
    'logging_level': 'Silent',
    'use_best_model': False,
    'ignored_features': ignored_features
}

### Обучение с валидацией

In [ ]:
%%time
model = CatBoostClassifier(**params)
model.fit(train_train_pool, eval_set=train_valid_pool)

CPU times: user 9min 6s, sys: 2min 7s, total: 11min 13s
Wall time: 2min 6s


### Визуализация результатов

In [ ]:
def visualize_learning_results(metrics):
    """
    Plot one graph for each metric

    :param metrics: dic of metrics
    """

    n = len(metrics)
    
    fig = make_subplots(
        rows=n,
        cols=1,
        subplot_titles=list(metrics.keys())
    )

    for i, (metric_name, curves) in enumerate(metrics.items()):
        for dataset_type, curve in curves.items():
            m = len(curve)
            fig.add_trace(
                go.Scatter(
                    x=np.arange(m),
                    y=curve,
                    mode='lines',
                    name=f'{dataset_type} {metric_name}'
                ),
                row=i + 1,
                col=1
            )

    fig.update_layout(
        title_text="Learning results",
        width=297. * 3,
        height=210. * 3 * n
    )
    fig.show()

In [ ]:
eval_results = model.get_evals_result()
metrics = dict(
    Logloss=dict(
        validation=eval_results['validation']['Logloss'],
        learn=eval_results['learn']['Logloss']
    ),
    AUC=dict(
        validation=eval_results['validation']['AUC']
    )
)

In [ ]:
visualize_learning_results(metrics)

### Объявление финальных параметров

In [ ]:
cat_features = [
    'ClientID',
    'ClientType',
    'CoachID',
    'GymID',
    'TrainingID',
    'Month',
    'Day',
    'DayOfWeek',
    'Season'
]
ignored_features = [
]
train_pool = Pool(
    pd.concat([X_train_train, X_train_valid]),
    pd.concat([y_train_train, y_train_valid]),
    cat_features=cat_features
)
params = {
    'iterations': 300,
    'learning_rate': 0.08,
    'logging_level': 'Silent',
    'use_best_model': False,
    'ignored_features': ignored_features
}
n_models = 10

### Обучение

In [ ]:
%%time
models = []
for i in range(n_models):
    model = CatBoostClassifier(**params)
    model.fit(train_pool)
    models.append(model)

CPU times: user 1h 12min 12s, sys: 18min 23s, total: 1h 30min 35s
Wall time: 14min 53s


### Получение предсказаний

In [ ]:
y_test_predicted = None
for model in models:
    predicted_probas = model.predict_proba(X_test)
    y_test_cur_predicted = pd.Series(
        predicted_probas[:, 1],
        index=X_test.index,
        name='Target'
    ).sort_index()
    
    if y_test_predicted is None:
        y_test_predicted = y_test_cur_predicted
    else:
        y_test_predicted += y_test_cur_predicted

y_test_predicted /= n_models

SUBMISSTION_FILE_NAME = 'Catboost_3_attemp_10_models_300_iters_0.8_lr.csv'

## Naive Bayes

In [ ]:
model = CategoricalNB()

In [ ]:
X_train_valid.TrainingID.isin(list(X_train_train.TrainingID.values)).describe()

count     64234
unique        2
top        True
freq      63939
Name: TrainingID, dtype: object

In [ ]:
existed_in_train_mask = X_train_valid.ClientID.isin(list(X_train_train.ClientID.values))
existed_in_train_mask &= X_train_valid.CoachID.isin(list(X_train_train.CoachID.values))
existed_in_train_mask &= X_train_valid.TrainingID.isin(list(X_train_train.TrainingID.values))

X_train_valid_existed = X_train_valid[existed_in_train_mask]
y_train_valid_existed = y_train_valid[existed_in_train_mask]

In [ ]:
model = model.fit(X_train_train.values, y_train_train.values)

In [ ]:
y_train_valid_probas = model.predict_proba(X_train_valid_existed.values)

In [ ]:
y_train_valid_probas

array([[0.00107237, 0.99892763],
       [0.00139774, 0.99860226],
       [0.00981249, 0.99018751],
       ...,
       [0.13281903, 0.86718097],
       [0.00404909, 0.99595091],
       [0.00374611, 0.99625389]])

In [ ]:
roc_auc_score(y_train_valid_existed.values, y_train_valid_probas[:,1])

0.7764905616335358

# Отправка результатов

## Определение пути

In [ ]:
submission_folder_path = os.path.join(PROJECT_DIR, 'submissions')
file_path = os.path.join(submission_folder_path, SUBMISSTION_FILE_NAME)
print(f"File will be saved to {file_path}")

File will be saved to ./submissions/Catboost_3_attemp_10_models_300_iters_0.8_lr.csv


## Сохранение

In [ ]:
y_test_predicted.to_csv(file_path)

## Отправка на кагл

In [ ]:
!kaggle competitions submit -c ml-posterior-gym-training-prediction -f "$file_path" -m "send $SUBMISSTION_FILE_NAME"

100%|███████████████████████████████████████| 1.09M/1.09M [00:06<00:00, 185kB/s]
Successfully submitted to ML Posterior. Gym training prediction

# Юнит тестирование


## Подключение библиотек

In [ ]:
import unittest

## Объявление тестирующего класса

In [ ]:
class TestNotebook(unittest.TestCase):
    def test_add(self):
        self.assertEqual(2 + 2, 4)

## Запуск тестов

In [ ]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_add (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK
